In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### done #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUDClass import CRUDMethod

###########################
# Data Manipulation / Model
###########################
# Done - update with your username and password and CRUD Python module name

username = "aacuser"
password = "wordpass13"

# Connect to database via CRUD Module
crud = CRUDMethod(username, password)


Connected to MongoDB


In [8]:
# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(crud.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

## Debug
print(len(df.to_dict(orient='records')))
print(df.columns)

10000
Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')


In [9]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#done Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#done Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Center(html.B(html.H1('Ariel Foor CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#done Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        className='row',
        style={'display': 'flex'},
        children=[
                html.Button(id='submit-button-one',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='submit-button-two',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='submit-button-three',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='submit-button-four', n_clicks=0, children='reset'),
                dash_table.DataTable(id='datatable-id'),
                html.Div(id='map-id')
        ]
        
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#done: Set up the features for your interactive data table to make it user-friendly for your client
                         
                        page_size=100,
                        style_table={'height':'300px','overflowY':'auto','overflowX':'auto'},
                        style_header={
                            'backgroundColor':'rgb(230,230,230)',
                            'fontWeight':'bold'        
                        },
                        style_data={
                            'whiteSpace':'normal',
                            'height':'auto'
                        },

            #tool tips
                        tooltip ={i: {
                        'value': i,
                        'use_with': 'both'}
                        for i in df.columns},
                        tooltip_delay=0,
                        tooltip_duration = None,

                #sorting features
                        sort_action='native',
                        sort_mode='multi',
                        filter_action='native',
                        editable=False,
                        column_selectable=False,
                        row_selectable='single',
                        row_deletable=False,
                        selected_rows=[],

                            ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6'),
                html.Div(
                    id='map-id',
                    className='col s12 m6')
        ])
])


In [10]:
#############################################
# Interaction Between Components / Controller
#############################################




@app.callback(
    [Output('datatable-id', 'data'), Output('map-id', 'children')],
    [Input('submit-button-one', 'n_clicks'),
     Input('submit-button-two', 'n_clicks'),
     Input('submit-button-three', 'n_clicks'),
     Input('submit-button-four', 'n_clicks'),
     Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")]
)
              

def update_dashboard(button1, button2, button3, button4, viewData, index):
## done Add code to filter interacti, ve data table with MongoDB queries
    ctx = dash.callback_context
    
    data = []
    map_children = html.Div("Map content here")

    if not ctx.triggered:
        button_id = 'No clicks yet'
    else:
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]


    if button_id in ['submit-button-one', 'submit-button-two', 'submit-button-three', 'submit-button-four']:
        if button_id ==  'submit-button-one':
            df = pd.DataFrame.from_records(crud.read({'$and': [ 
                {'$or': [ {'breed':'Labrador Retriever Mix'}, 
                        {'breed':'Chesapeake Bay Retriever'},
                        {'breed':'Newfoundland'}]}, 
                {'sex_upon_outcome':'Intact Female'}, 
                {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))


        elif button_id == 'submit-button-two':
            df = pd.DataFrame.from_records(crud.read({'$and': [ 
                {'$or': [ {'breed':'German Shepherd'}, 
                        {'breed':'Alaskan Malamute'},
                        {'breed':'Old English Sheepdog'},
                        {'breed':'Siberian Husky'},
                        {'breed':'Rottweiler'}]}, 
                {'sex_upon_outcome':'Intact Male'}, 
                {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))


        elif button_id == 'submit-button-three':
            df = pd.DataFrame.from_records(crud.read({'$and': [ 
                {'$or': [ {'breed':'Doberman Pinscher'}, 
                        {'breed':'German Sheperd'},
                        {'breed':'Golden Retriever'},
                        {'breed':'Bloodhound'},
                        {'breed':'Rottweiler'}]}, 
                {'sex_upon_outcome':'Intact Male'}, 
                {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
            bt1, bt2, bt4 = 0

        elif button_id == 'submit-button-four':
            df = pd.DataFrame.from_records(crud.read())

            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
            data=df.to_dict('records')  
            
            
    elif button_id == 'datatable-id':
        if viewData is None or index is None:
            return [], html.Div("Map content here")

        dff = pd.DataFrame.from_dict(viewData)
        row = index[0] if index else 0

        map_children = [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                    dl.Tooltip(dff.iloc[row, 4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[row, 9])
                    ])
                ])
            ])
        ]

            
        return data, map_children

In [11]:
# Display the breeds of animal based on quantity represented in
# the data table


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    df = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
            figure = px.pie(df, names='breed', title='Preferred Animals')
        )    
    ]


In [12]:
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

app.run_server(debug=True)

TypeError: an integer is required (got type NoneType)

In [ ]:
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable


# @app.callback(Output('map-id', "children"), 
#               [Input('datatable-id', "derived_viewport_data"),
#               Input('datatable-id',"derived_viewport_selected_rows")])

# def update_map(viewData, index):  
#     if viewData is None:
#         return
#     elif index is None:
#         return
    
#     dff = pd.DataFrame.from_dict(viewData)
#     # Because we only allow single row selection, the list can be converted to a row index here
#     if index is None:
#         row = 0
#     else: 
#         row = index[0]
        
#     # Austin TX is at [30.75,-97.48]
#     return [
#         dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
#             dl.TileLayer(id="base-layer-id"),
#             # Marker with tool tip and popup
#             # Column 13 and 14 define the grid-coordinates for the map
#             # Column 4 defines the breed for the animal
#             # Column 9 defines the name of the animal
#             dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
#                 dl.Tooltip(dff.iloc[row,4]),
#                 dl.Popup([
#                     html.H1("Animal Name"),
#                     html.P(dff.iloc[row,9])
#                 ])
#             ])
#         ])
#     ]



